In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from http.cookiejar import CookieJar
from urllib.request import build_opener, HTTPCookieProcessor

In [4]:
a_dataframe = pd.read_csv("C:/Users/yttn0/Desktop/git/GameChanger_my/dataset/games.csv")
a_dataframe

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,2159120,SUR5,"Apr 1, 2023",0 - 0,0,0,0.00,0,"Innovative Gameplay SUR5 brings a brand new, f...","['English', 'Polish', 'Spanish - Latin America']",...,0,0,0,Katharsis,Katharsis,"Single-player,Multi-player,PvP,Online PvP,Stea...","Free to Play,Indie,Massively Multiplayer,Strat...",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71712,2125160,Prison Life 2,"Mar 30, 2023",0 - 20000,6,0,10.79,0,Core Features Real players only! All Guards an...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,AO Games,AO Games,"Single-player,Multi-player,PvP,Online PvP,Co-o...","Casual,Indie,RPG,Simulation,Strategy","First-Person,Detective,Social Deduction,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71713,2318860,Architecture Zeitgeist,"Mar 27, 2023",0 - 20000,0,0,5.99,0,Architecture Zeitgeist is a game created with ...,"['English', 'Japanese', 'Portuguese - Brazil',...",...,0,0,0,Matheus Rudo,Rudo Studios,Single-player,"Casual,Simulation,Early Access","Casual,Simulation,Puzzle,Exploration,Immersive...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71714,2180330,Girl's Way to Survive 2: Bloody Museum,"Apr 2, 2023",0 - 0,0,0,2.39,0,"Development progress: prologue map (finished),...",['Simplified Chinese'],...,0,0,0,绅士游戏工坊,绅士游戏工坊,Single-player,"Action,Casual,Indie,RPG,Early Access",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [5]:
game_no = a_dataframe['AppID']
# print(a_dataframe.loc[60445])

In [ ]:
# first_idx - 탐색 시작할 인덱스 번호  result - 결과값 담을 배열
def start_crawling(first_idx, result):
    driver = webdriver.Chrome('C:/Users/yttn0/My_Python/WebDriver/chromedriver.exe') # chromedriver 연결
    ## short discription
    ## detailed discription
    for no in range(first_idx, len(game_no)+1):
        ### URL 가져오기 시작 ###
        url = 'https://store.steampowered.com/app/%d' %(int(game_no[no]))
        print(url)
        driver.get(url)
        time.sleep(3)
        html = urllib.request.urlopen(url)
        driver.execute_script("window.scrollTo(0, 5000);") # 스크롤 내린 상태로 Parsing 진행
        time.sleep(10)
        html = driver.page_source
        SoupUrl = BeautifulSoup(html, 'html.parser')
        ### URL 가져오기 끝 ###
        
        tag_name_overall_reviews = SoupUrl.find('div', attrs={'id':'review_histogram_rollup_section'}) # Overall Reviews 정보
        tag_name_recent_reviews = SoupUrl.find('div', attrs={'id':'review_histogram_recent_section'}) # Recent Reviews 정보

    ### Overall Review 및 recent_review 데이터 추출 시작 ###
    try:
        overall_reviews = tag_name_overall_reviews.find('span', attrs={'class':'game_review_summary'})
        overall_reviews = overall_reviews.get_text()

        recent_reviews = tag_name_recent_reviews.find('span', attrs={'class':'game_review_summary'})
        recent_reviews = recent_reviews.get_text()
    except:
        print("Overall Review 또는 Recent Review 데이터가 없습니다.")
    ### Overall Review 및 recent_review 데이터 추출 끝 ###

    ### 리뷰 부분 시작 ###
    reviews_total = []
    try:
      # 리뷰있는 부분 찾기
        tag_name_most_helpful_reviews = SoupUrl.find('div', attrs={'id':'Reviews_summary', 'class':'user_reviews_container'})    
        most_helpful_reviews = tag_name_most_helpful_reviews.select("div>div.leftcol>div.review_box")

        for c_review in most_helpful_reviews:
            rec = c_review.find("div", attrs={'class':'title ellipsis'}).get_text() # 추천 or 비추천
            content = c_review.find("div", attrs={'class':'content'}).get_text() # 리뷰 내용

            customer_review = rec + ", "  + content
            reviews_total.append(customer_review)
    except:
        print("리뷰가 없습니다.")
    ### 리뷰 부분 종료 ###
    
    ############ Metacritic ##############
    ### metacritic URL 주소 추출 시작 ###
    try:
        metacritic_url = SoupUrl.find('div', attrs={'id':'game_area_metalink'})
        metacritic_url = metacritic_url.find('a')['href']
        print(metacritic_url)
        
#         driver = webdriver.Chrome('C:/Users/yttn0/My_Python/WebDriver/chromedriver.exe') # chromedriver 연결
#         url = "https://www.metacritic.com/game/pc/stardew-valley"
#         print(url)

        try:
            driver.get(metacritic_url)
            time.sleep(3)
            html = driver.page_source
            # 리디랙션 루프 방지
            cj = CookieJar()
            opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
            SoupUrl = BeautifulSoup(html, 'html.parser')

            ### metacritic - summary #### (완)
            metacritic_summary = SoupUrl.find('li', {'class':'summary_detail product_summary'})
            try:
                meta_summary = metacritic_summary.find('span', attrs={'class':'blurb blurb_expanded'}).get_text() 
            except:
                meta_summary = metacritic_summary.find("span", {'class':'data'}).get_text() # 스크립트수가 많은 경우

            print('summary: ' + meta_summary)
            print('---------------------------------')
            ### summary 끝 ####

            ### metacritic - genre #### (완)
            metacritic_genre = SoupUrl.find('li', attrs={'class':'summary_detail product_genre'})
            try:
                find_genre = metacritic_genre.find_all('span', attrs = {'class': 'data'})# 모든 장르
                meta_genre  = ''
                for i in find_genre:
                    meta_genre = meta_genre + i.get_text() + ' '
        #             print(i.get_text())
            except:
                print('genre 데이터가 없습니다')

            print('genre: ' + meta_genre)
            print('--------------------------------')
            ### genre 끝 ####

            ### metacritic - critic review ####

            meta_creview = ''
            try:
                driver.find_element(By.CSS_SELECTOR,"li.nav.nav_critic_reviews > span > span > a").click() # critic review로 이동
        #         driver.find_element(By.CSS_SELECTOR,".see_all > a").click() # critic review로 이동
                metacritic_creview = SoupUrl.find('ol', attrs = {'class':'reviews critic_reviews'})

                # 모든 리뷰(first, last 제외)


                ###### 모든 리뷰가 뽑히지 않음!!!!#####
                find_creview = metacritic_creview.find_all('li', attrs = {'class': 'review critic_review'}) ### error#####
                print(len(find_creview))

                for i in find_creview:
                    meta_creview = meta_creview + i.find('div', attrs = {'class': 'review_body'}).get_text()
        #             print(i.find('div', attrs = {'class': 'review_body'}).get_text())

                # fist, last 리뷰(완)
                # first
                find_f_creview = metacritic_creview.find('li', attrs = {'class': 'review critic_review first_review'})
                meta_creview = meta_creview + find_f_creview.find('div', attrs = {'class': 'review_body'}).get_text()
                #last
                find_l_creview = metacritic_creview.find('li', attrs = {'class': 'review critic_review last_review'})
                meta_creview = meta_creview + find_l_creview.find('div', attrs = {'class': 'review_body'}).get_text()

                print(meta_creview)
                print('----------------------------------------------')
            except:
                print('critic review가 없습니다')
            ### \critic review 끝 ####

            ### metacritic - user review ####
            meta_ureview = ''

        #     pagenum = SoupUrl.find('li', attrs = {'class':'page last_page'}).find('a').get_text()
        #     print(pagenum)


            try:
                driver.find_element(By.CSS_SELECTOR,"li.nav.nav_user_reviews > span > span > a").click() # critic review로 이동
                metacritic_ureview = SoupUrl.find('ol', attrs = {'class':'reviews user_reviews'})

                # 모든 리뷰(first, last 제외)

                ###### 모든 리뷰가 뽑히지 않음!!!!#####
                find_ureview = metacritic_ureview.find_all('li', attrs = {'class': 'review user_review'}) ### error#####
                print(len(find_ureview))

                for i in find_ureview:
                    try: # expand
                        meta_ureview = meta_ureview + i.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text()
                        print(i.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text())
                    except: # expand 없는 경우
                        meta_ureview = meta_ureview + i.find('div', attrs = {'class': 'review_body'}).find('span').get_text()
                        print(i.find('div', attrs = {'class': 'review_body'}).find('span').get_text())
                    print('-------------------------------------------------------')

        #         fist, last 리뷰(완)
                #first
                find_f_ureview = metacritic_ureview.find('li', attrs = {'class': 'review user_review first_review'})
                try: # expand
                    meta_ureview = meta_ureview + find_f_ureview.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text()
                    print('first: '+ find_f_ureview.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text())
                except: # expand 없는 경우
                    meta_ureview = meta_ureview + find_f_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text()
                    print('first: '+find_f_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text())

                #last
                find_l_ureview = metacritic_ureview.find('li', attrs = {'class': 'review user_review last_review'})
                meta_ureview = meta_ureview + find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text()
                try: # expand
                    meta_ureview = meta_ureview + find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text()
        #             print('last: ' +find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text())
                except: # expand 없는 경우
                    meta_ureview = meta_ureview + find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text()
        #             print('last: ' + find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text())

        #         print(meta_ureview)
            except:
                print('user review가 없습니다')
            ### \critic review 끝 ####

        except:
            print("url 실패")
    except:
        print("Metacritic 링크가 존재하지 않습니다.")
            ### metacritic URL 주소 추출 끝 ###

    steam_value = [overall_reviews] + [recent_reviews] + [metacritic_url] + [reviews_total]
    print('steam_value: '+ steam_value)
    result.append(steam_value)
    
    meta_value = [metacritic_summary] + [metacritic_genre] + [meta_creview] + [meta_ureview]
    print('meta_Value: ' + meta_value)
    result.append(meta_value)
        


In [ ]:
result=[]
start_crawling(0, result) # 임시로 0번째 인덱스(처음)부터 시작하게끔 설정한 상태
Wcrawling_tbl = pd.DataFrame(result, columns=('Overall Reviews', 'Recent Reviews', 'Most Helpful Reviews', 'Currently popular'))
Wcrawling_tbl.to_csv('CUsersravengitgameChangerdatasetsteamGames.csv', encoding='cp949', mode='w', index=True)
del result[:]

In [6]:
############ Metacritic ###############
from http.cookiejar import CookieJar
from urllib.request import build_opener, HTTPCookieProcessor

driver = webdriver.Chrome('C:/Users/yttn0/My_Python/WebDriver/chromedriver.exe') # chromedriver 연결
url = "https://www.metacritic.com/game/pc/stardew-valley"
print(url)

try:
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    # 리디랙션 루프 방지
    cj = CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    SoupUrl = BeautifulSoup(html, 'html.parser')
    
    ### metacritic - summary #### (완)
    metacritic_summary = SoupUrl.find('li', {'class':'summary_detail product_summary'})
    try:
        meta_summary = metacritic_summary.find('span', attrs={'class':'blurb blurb_expanded'}).get_text() 
    except:
        meta_summary = metacritic_summary.find("span", {'class':'data'}).get_text() # 스크립트수가 많은 경우
        
    print('summary: ' + meta_summary)
    print('---------------------------------')
    ### summary 끝 ####

    ### metacritic - genre #### (완)
    metacritic_genre = SoupUrl.find('li', attrs={'class':'summary_detail product_genre'})
    try:
        find_genre = metacritic_genre.find_all('span', attrs = {'class': 'data'})# 모든 장르
        meta_genre  = ''
        for i in find_genre:
            meta_genre = meta_genre + i.get_text() + ' '
#             print(i.get_text())
    except:
        print('genre 실패')

    print('genre: ' + meta_genre)
    print('--------------------------------')
    ### genre 끝 ####
    
    ### metacritic - critic review ####
            
    meta_creview = ''
    try:
        driver.find_element(By.CSS_SELECTOR,"li.nav.nav_critic_reviews > span > span > a").click() # critic review로 이동
#         driver.find_element(By.CSS_SELECTOR,".see_all > a").click() # critic review로 이동
        metacritic_creview = SoupUrl.find('ol', attrs = {'class':'reviews critic_reviews'})
        
        # 모든 리뷰(first, last 제외)
        

        ###### 모든 리뷰가 뽑히지 않음!!!!#####
        find_creview = metacritic_creview.find_all('li', attrs = {'class': 'review critic_review'}) ### error#####
        print(len(find_creview))

        for i in find_creview:
            meta_creview = meta_creview + i.find('div', attrs = {'class': 'review_body'}).get_text()
#             print(i.find('div', attrs = {'class': 'review_body'}).get_text())
            
        # fist, last 리뷰(완)
        # first
        find_f_creview = metacritic_creview.find('li', attrs = {'class': 'review critic_review first_review'})
        meta_creview = meta_creview + find_f_creview.find('div', attrs = {'class': 'review_body'}).get_text()
        #last
        find_l_creview = metacritic_creview.find('li', attrs = {'class': 'review critic_review last_review'})
        meta_creview = meta_creview + find_l_creview.find('div', attrs = {'class': 'review_body'}).get_text()
        
        print(meta_creview)
        print('----------------------------------------------')
    except:
        print('critic review가 없음')
    ### \critic review 끝 ####
    
    ### metacritic - user review ####
    meta_ureview = ''
    
#     pagenum = SoupUrl.find('li', attrs = {'class':'page last_page'}).find('a').get_text()
#     print(pagenum)

    
    try:
        driver.find_element(By.CSS_SELECTOR,"li.nav.nav_user_reviews > span > span > a").click() # critic review로 이동
        metacritic_ureview = SoupUrl.find('ol', attrs = {'class':'reviews user_reviews'})
        
        # 모든 리뷰(first, last 제외)
        
        ###### 모든 리뷰가 뽑히지 않음!!!!#####
        find_ureview = metacritic_ureview.find_all('li', attrs = {'class': 'review user_review'}) ### error#####
        print(len(find_ureview))

        for i in find_ureview:
            try: # expand
                meta_ureview = meta_ureview + i.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text()
                print(i.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text())
            except: # expand 없는 경우
                meta_ureview = meta_ureview + i.find('div', attrs = {'class': 'review_body'}).find('span').get_text()
                print(i.find('div', attrs = {'class': 'review_body'}).find('span').get_text())
            print('-------------------------------------------------------')
            
#         fist, last 리뷰(완)
        #first
        find_f_ureview = metacritic_ureview.find('li', attrs = {'class': 'review user_review first_review'})
        try: # expand
            meta_ureview = meta_ureview + find_f_ureview.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text()
            print('first: '+ find_f_ureview.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text())
        except: # expand 없는 경우
            meta_ureview = meta_ureview + find_f_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text()
            print('first: '+find_f_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text())
        
        #last
        find_l_ureview = metacritic_ureview.find('li', attrs = {'class': 'review user_review last_review'})
        meta_ureview = meta_ureview + find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text()
        try: # expand
            meta_ureview = meta_ureview + find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text()
#             print('last: ' +find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span', attrs = {'class':'blurb blurb_expanded'}).get_text())
        except: # expand 없는 경우
            meta_ureview = meta_ureview + find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text()
#             print('last: ' + find_l_ureview.find('div', attrs = {'class': 'review_body'}).find('span').get_text())
        
#         print(meta_ureview)
    except:
        print('user review가 없음')
    ### \critic review 끝 ####
    
except:
    print("url 실패")
       

<ipython-input-6-628a90ba6756>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/yttn0/My_Python/WebDriver/chromedriver.exe') # chromedriver 연결


https://www.metacritic.com/game/pc/stardew-valley
summary: Stardew Valley is an RPG-style farming game. It's up to you to choose your path... Will you devote your time to raising crops and caring for animals? Will you spend time in the local village, befriending the local townspeople? Do you get a kick out of slaying dreadful monsters in the mysterious mountain caverns? Or will you try to accomplish everything and claim the coveted title of Stardew Hero?
---------------------------------
genre: General Role-Playing General 
--------------------------------
5

                                Stardew Valley has been the most rich and heartwarming experience I’ve had in a game in years.
                            
                                Stardew Valley is an absolutely charming little gem about forgetting all your worries and relaxing on a farm.
                            
                                Don't start this game if you value relationships, your job, your social lif